In [328]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [329]:
# BUKA CHROME
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(options=chrome_options)

In [330]:
import time

# ARAHKAN CHROME KE LINK
driver.get('https://pasanglaut.com/id/jakarta/tanjung-priok#_tide_table')

# Menambah waktu tunggu selama 5 detik
time.sleep(15)

In [331]:
# tampilkan hasil scrapping dan rapihkan dengan beautifulsoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
# print(soup.prettify())

Get Tanggal

In [332]:
# Cari semua tag <tr> dengan class yang sesuai dan memiliki atribut "onclick"
tr_elements = soup.find_all('tr', class_='tabla_mareas_fila', onclick=True)

# Loop melalui setiap elemen <tr>\
tanggal = []
for tr in tr_elements:
    # Dapatkan nilai atribut "title" pada elemen <tr>
    title = tr.get('title')
    pisah = title.split(",")
    tanggal.append(pisah[1][1:])
    # Tampilkan nilai atribut "title"
    # print(title)

jumlah_hari = int((len(tanggal))/3)
tanggal = tanggal[:jumlah_hari]
# tanggal

Get Pasang Surut

In [333]:
# Cari semua tag <tr> dengan class yang sesuai dan memiliki atribut "onclick"
tr_elements = soup.find_all('tr', class_='tabla_mareas_fila', onclick=True)

# Loop melalui setiap elemen <tr>
pasang_surut = []
pasang_surut_sementara = []
jam_pasang = []
jam_surut = []
tinggi_pasang = []
tinggi_surut = []
for tr in tr_elements:
    td_elements = tr.find_all('td', class_='tabla_mareas_marea tabla_mareas_marea_border_bottom')

    # Loop melalui elemen-elemen <td> yang ditemukan dan ambil isi teksnya
    i_td_elements = 0
    for td in td_elements:
        i_td_elements += 1
        # Dapatkan semua tag <div> di dalam elemen <td>
        div_elements = td.find_all('div')
        
        # Loop melalui setiap elemen <div>
        i_div_elements = 0
        for div in div_elements:
            i_div_elements += 1
            # Cek apakah elemen <div> memiliki class "tabla_mareas_marea_hora"
            if 'tabla_mareas_marea_hora' in div.get('class', []):
                # Tampilkan teks elemen <div> dengan class "tabla_mareas_marea_hora"
                jam = div.get_text()
                # print(i_td_elements,".",i_div_elements,". Jam:", jam)
                if i_td_elements == 1 :
                    jam_pasang.append(jam)
                elif i_td_elements == 2 : 
                    jam_surut.append(jam)
            
            # Cek apakah elemen <div> memiliki class "tabla_mareas_marea_altura"
            if 'tabla_mareas_marea_altura' in div.get('class', []):
                # Tampilkan teks elemen <div> dengan class "tabla_mareas_marea_altura"
                tinggi = (div.get_text()).replace('\n','')
                tinggi = tinggi.replace(" ","")
                tinggi = float(tinggi.replace("m",""))
                # print(i_td_elements,".",i_div_elements,". Tinggi:", tinggi)
                if i_td_elements == 1 :
                    tinggi_pasang.append(tinggi)
                elif i_td_elements == 2 : 
                    tinggi_surut.append(tinggi)
                    
jam_pasang = jam_pasang[:jumlah_hari]
jam_surut = jam_surut[:jumlah_hari]
tinggi_pasang = tinggi_pasang[:jumlah_hari]
tinggi_surut = tinggi_surut[:jumlah_hari]


Get Koefisien

In [334]:
# Cari semua tag <td> dengan class yang sesuai
td_elements = soup.find_all('td', class_='tabla_mareas_coeficiente')

koefisien = []
# Loop melalui setiap elemen <td>
for td in td_elements:
    # Cari tag <div> dengan class "tabla_mareas_coeficiente_altura" di dalam elemen <td>
    div_element_string = td.find('div', class_='tabla_mareas_coeficiente_altura')
    div_element_int = td.find('div', class_='tabla_mareas_coeficiente_numero')
    # Periksa apakah tag <div> ditemukan
    if div_element_string :
        # Dapatkan teks dari tag <div>
        div_text_angka = div_element_string.get_text()
        div_text_angka = str(div_text_angka)
        # Tampilkan teks tag <div>
        # print(div_text_angka)
    
    if div_element_int :
        # Dapatkan teks dari tag <div>
        div_text_string = div_element_int.get_text()
        
        # Tampilkan teks tag <div>
        div_text_string = div_text_string.split(' ')
        div_text_string = str(div_text_string[1])
        # print(div_text_string)
        
    koefisien.append(div_text_string+" "+div_text_angka)
    
koefisien = koefisien[:jumlah_hari]
    


In [337]:
import pandas as pd

# Membuat DataFrame dari list-list tersebut
data = {
    'Tanggal': tanggal,
    'Jam Pasang': jam_pasang,
    'Tinggi Pasang': tinggi_pasang,
    'Jam Surut': jam_surut,
    'Tinggi Surut': tinggi_surut,
    'Koefisien': koefisien
}

df = pd.DataFrame(data)

df

,Tanggal,Jam Pasang,Tinggi Pasang,Jam Surut,Tinggi Surut,Koefisien
0,1 Desember 2020,10:05,1.0,21:24,0.2,77 tinggi
1,2 Desember 2020,10:20,1.0,22:09,0.2,74 tinggi
2,3 Desember 2020,10:39,1.0,22:49,0.2,69 rata-rata
3,4 Desember 2020,11:01,1.0,23:24,0.2,62 rata-rata
4,5 Desember 2020,11:24,1.0,23:52,0.2,56 rata-rata
5,6 Desember 2020,11:43,1.0,11:41,0.9,51 rata-rata
6,7 Desember 2020,0:13,0.2,10:37,0.8,50 rata-rata
7,8 Desember 2020,0:26,0.3,9:22,0.8,55 rata-rata
8,9 Desember 2020,0:32,0.3,8:49,0.9,64 rata-rata
9,10 Desember 2020,0:28,0.4,8:44,1.0,74 tinggi


Gabungkan Data

In [338]:
df_panggil = pd.read_csv('pasang_surut_2020.csv')

# Menggabungkan kedua DataFrame
merged_df = pd.concat([df_panggil, df])

merged_df = merged_df.drop(merged_df.columns[0], axis=1)
merged_df


,Tanggal,Jam Pasang,Tinggi Pasang,Jam Surut,Tinggi Surut,Koefisien
0,1 Januari 2020,10:29,0.9,23:38,0.3,48 rendah
1,2 Januari 2020,9:53,0.8,23:42,0.4,43 rendah
2,3 Januari 2020,9:19,0.8,23:28,0.4,42 rendah
3,4 Januari 2020,8:53,0.9,22:17,0.4,43 rendah
4,5 Januari 2020,8:39,0.9,18:06,0.4,48 rendah
...,...,...,...,...,...,...
26,27 Desember 2020,9:14,1.0,19:53,0.2,67 rata-rata
27,28 Desember 2020,9:25,1.0,20:30,0.2,72 tinggi
28,29 Desember 2020,9:40,1.1,21:09,0.2,76 tinggi
29,30 Desember 2020,9:57,1.1,21:24,0.2,78 tinggi


In [339]:
merged_df.to_csv("pasang_surut_2020.csv")